# Kernel chido 

In [1]:
using CUDA, LinearAlgebra, Statistics

In [2]:
A = [3 -2 0; 1 3 -1; 2 1 4]
b = [-3, -3, 6]
#A\b = [-1 0 2]

M = Diagonal(A) # Tal vez esta es una mejor forma(?)

N = M - A

T = inv(M) * N

f = inv(M) * b

nT, mT = size(T)

S = zeros(Int64, nT)

[S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]

P = zeros(nT, mT)
[P[i,j] = 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0];
#0.0  1.0  0.0
#0.5  0.0  0.5
#0.5  0.5  0.0

[P[i,j] = sum(P[i,1:j]) for i = 1:nT, j = 1:mT if P[i,j] != 0];
#0.0 1.0 0.0
#0.5 0.0 1.0
#0.5 1.0 0.0

In [3]:
T

3×3 Array{Float64,2}:
  0.0        0.666667  0.0
 -0.333333   0.0       0.333333
 -0.5       -0.25      0.0

In [4]:
P

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 0.5  0.0  1.0
 0.5  1.0  0.0

In [44]:
# El tamaño de la cadena
e = 10
# Aquí cambiamos la posición del vector solución que queremos aproximar, ya sea X[1], X[2] o X[3]
var = 1
# Calculamos point y nextpoint para así ya no tener que meter el número aleatorio, por ahora. 
u = rand(e)
Point = [var]
Nextpoint = []
for tamaño = 1:e
    np = 1
    while u[tamaño] >= P[Point[tamaño], np]
        np += 1
    end
    push!(Nextpoint, np)
    push!(Point, np)
end

d_W = CUDA.ones(Float32, e) # Aquí suponemos W_0 = 1.
d_point = CuArray(Point)
d_nextpoint = CuArray{Int32}(Nextpoint)
d_W_new = CUDA.ones(Float32, e)
d_P = CuArray{Float32}(P)
d_T = CuArray{Float32}(T)
#d_X = CUDA.fill(Float32(f[var]), e) # Algo así hay que hacer cuando hagamos MUCHAS cadenas a la vez. 
d_X = CUDA.zeros(Float32, e)
d_f = CuArray{Float32}(f)
d_flag = CUDA.fill(Inf32, e)

# Con este kernel construiremos por ahora UNA cadena. 
function kernel(W, W_new, X, T, P, nextpoint, point, f, flag)
    i = (blockIdx().x-1) * blockDim().x + threadIdx().x
    # Esto de elevarlo está mal! No son la misma constante para cada iteración!
    W_new[i] = W[i] * CUDA.pow((T[point[i], nextpoint[i]]/P[point[i], nextpoint[i]]), i)
    # Todo esto que sigue es para ver en qué momento los pesos de la cadena son menor a la epsilon que demos
    for h in 1:length(W_new) 
        if abs(W_new[i]) < 0.005
            #@cuprintln("i = $i")
            flag[1] = i
        end
            break
    end
    # Esto para que se hagan en orden tanto como se pueda (Raro, no es tan paralelo entonces?)
    sync_threads()
    if i < flag[1]
        @cuprintln("W_new[$i] = $(W_new[i])")
        X[i] +=  W_new[i] * f[nextpoint[i]]
    end
    return
end

@cuda threads = e kernel(d_W, d_W_new, d_X, d_T, d_P, d_nextpoint, d_point, d_f, d_flag)

# Se toma únicamente el promedio de los valores hasta cuando los pesos fueron más pequeñós que la epsilon
sum(Array(d_X)[1:Int(Array(d_flag)[1])-1]) + f[var]

W_new[1] = 0.666667
W_new[2] = 0.444444
W_new[3] = 0.296296
W_new[4] = 0.012346
W_new[5] = -1.000000
W_new[6] = 0.087792


-1.4766804575920105

In [7]:
d_X

10-element CuArray{Float32,1}:
 -1.6666667
 -1.4444444
 -1.2962964
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0

In [9]:
Int(Array(d_flag)[1])

4